# Notebook to Visualize and Analyze Stocks
By Ricardo Saca 

## Import Packages

In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from datetime import date
from dateutil.relativedelta import relativedelta


In [14]:
#Ask for user for ticker
ticker = input('Please input stock ticker:')
print(ticker)

SBUX


In [15]:
#get ticker with yahoofinance api 
stock = yf.Ticker(ticker)

## General information of Stock

In [16]:
print(stock.info['shortName'],'\n')
print(stock.info['sector'],'\n')
print(stock.info['longBusinessSummary'])

Starbucks Corporation 

Consumer Cyclical 

Starbucks Corporation, together with its subsidiaries, operates as a roaster, marketer, and retailer of specialty coffee worldwide. The company operates through three segments: Americas, International, and Channel Development. Its stores offer coffee and tea beverages, roasted whole bean and ground coffees, single-serve and ready-to-drink beverages, and iced tea; and various food products, such as pastries, breakfast sandwiches, and lunch items. The company also licenses its trademarks through licensed stores, and grocery and foodservice accounts. The company offers its products under the Starbucks, Teavana, Seattle's Best Coffee, Evolution Fresh, Ethos, Starbucks Reserve, and Princi brand names. As of October 29, 2020, it operated approximately 32,000 stores. Starbucks Corporation was founded in 1971 and is based in Seattle, Washington.


### Earnings Calendar

In [17]:
stock.calendar

,0,1
Earnings Date,2021-10-27 00:00:00,2021-11-01 00:00:00
Earnings Average,0.77,0.77
Earnings Low,0.7,0.7
Earnings High,0.84,0.84
Revenue Average,7239360000,7239360000
Revenue Low,6388700000,6388700000
Revenue High,7512600000,7512600000


### Recent Stock recommendations last two months

In [18]:
recommendations = stock.recommendations.reset_index()
two_months = pd.to_datetime((date.today() + relativedelta(months=-2)))

#show last two months 
recommendations[recommendations['Date'] >= two_months]

,Date,Firm,To Grade,From Grade,Action
280,2021-07-16 12:58:41,Oppenheimer,Outperform,,main
281,2021-07-23 10:11:57,Baird,Outperform,Neutral,up
282,2021-07-27 09:46:33,Guggenheim,Neutral,,init
283,2021-07-28 12:03:42,BMO Capital,Outperform,,main
284,2021-07-28 13:35:52,Morgan Stanley,Equal-Weight,,main
285,2021-07-28 15:17:32,Piper Sandler,Neutral,,main
286,2021-07-28 15:18:51,Barclays,Overweight,,main


## Stock Visualizations 

In [19]:
period = input('What period are you looking for? (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)')

In [20]:
# Get stock data 
data = yf.download(ticker, period=period, interval="1d")

[*********************100%***********************]  1 of 1 completed


In [21]:
# Reset index 
data = data.reset_index()

In [23]:
#Create chart
fig = go.Figure(data=[go.Candlestick(x=data['Date'], 
                                    open=data['Open'],
                                    high=data['High'],
                                    low=data['Low'], 
                                    close=data['Close'])])

#Change title
fig.update_layout(
    title= f"{stock.info['shortName']} ({ticker}) Candlestick Chart",
    font=dict(
        family="Times New Roman",
        size=15
    )
)

#Show chart 
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [24]:
fig = px.line(data,
                x='Date', 
                y='Open', 
                title=f"{stock.info['shortName']} ({ticker}) Stock Prices"
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
fig = go.Figure(data=go.Ohlc(x=data['Date'],
                            open=data['Open'],
                            high=data['High'],
                            low=data['Low'],
                            close=data['Close']))
#Change title
fig.update_layout(
    title= f"{stock.info['shortName']} ({ticker}) OHLC Chart",
    font=dict(
        family="Times New Roman",
        size=15
    )
)

fig.show()

## Ratio Analysis

### Get Data

In [ ]:
#Formula to clean financial statments from SEC 
def clean_data(data):
    data = data.copy()
    #identify columns and year
    columns = list(data.loc[1])
    #Deal with format change in 2018 statements
    if columns[1] is np.nan:
        columns = list(data.loc[2])
        for index, col in enumerate(columns):
            if type(col) == 'float':
                pass
                columns[index] = col.replace(u'\xa0', u' ')
    columns[0] = 'Account'
    #rename columns 
    data.columns = columns
    #drop empty columns 
    data = data.dropna()
    #drop title column 
    data.drop(data.index[0], inplace=True)
    #reset index 
    data = data.reset_index(drop=True)
    #clean negative numbers
    for col in data:
        if col == 'Account':
            pass
        else:
            data[col] = data[col].map(lambda x: x.replace('(','-').replace(')','').replace(',','').replace('—','0'))
    #convert to float
    dates = columns[1:]
    data[dates] = data[dates].astype(float)
    #set account as index 
    data.set_index('Account', inplace=True)
    #return clean data
    return data 

Getting 2020 Data from SEC 

In [ ]:
#Fetching financial data from SEC for SBUX 
#2020_url 
url = "https://www.sec.gov/Archives/edgar/data/0000829224/000082922420000078/sbux-20200927.htm"

#read tables of url file 
sec = pd.read_html(url)

Getting 2018 data from SEC 

In [ ]:
#2018 url (in order to get 3 yrs of data in analysis, also need 2017 for pct change) 
url_2018 = "https://www.sec.gov/Archives/edgar/data/829224/000082922418000052/sbux-9302018x10xk.htm#s67ECBBF7328F565DB062A56D81B66975"

sec2018 = pd.read_html(url_2018) 

In [ ]:
# Find what tables we care about
# count = 0
# for i in sec2018:
#     count +=1
#     print(f'\n+++++++++++++++++++++ {count} +++++++++++++++\n')         
#     print(i.head())
#     if (count > 32  ) & (count < 37):
#         print(f'\n+++++++++++++++++++++ {count} +++++++++++++++\n')         
#         print(i.head())

#for 2020 
#earnings = 32 | income = 33 | balsht = 34 | cashflw = 35 
# for 2018 
#earnings = 33 | income = 34 | balsht = 35 | cashflw = 36 

In [ ]:
#2020
earnings20 = sec[32][[2,4,10,16]].copy()
inc20 = sec[33][[2,4,10,16]].copy()
balsht20 = sec[34][[2,4,10]].copy()
cf20 = sec[35][[2,4,10,16]].copy()

#2018 
earnings18 = sec2018[132][[0,2,6,10]].copy()
inc18 = sec2018[133][[0,2,6,10]].copy()
balsht18 = sec2018[134][[0,2,6]].copy()
cf18 = sec2018[135][[0,2,6,10]].copy()

In [ ]:
#Clean Data 
earnings20 = clean_data(earnings20)
inc20 = clean_data(inc20)
balsht20 = clean_data(balsht20)
cf20 = clean_data(cf20)
earnings18 = clean_data(earnings18) 
inc18 = clean_data(inc18)
balsht18 = clean_data(balsht18)
cf18 = clean_data(cf18)

In [ ]:
writer = pd.ExcelWriter('SBUX Financials.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
earnings20.to_excel(writer, sheet_name='earnings 20')
inc20.to_excel(writer, sheet_name='income 20')
balsht20.to_excel(writer, sheet_name='bal sht 20')
cf20.to_excel(writer, sheet_name='cf 20')
earnings18.to_excel(writer, sheet_name='earnings 18')
inc18.to_excel(writer, sheet_name='income 18')
balsht18.to_excel(writer, sheet_name='bal sht 18')
cf18.to_excel(writer, sheet_name='cf 18')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

Merging data sets 

In [ ]:
#merging
earnings = earnings20.copy()
earnings['Oct 01, 2017'] = earnings18['Oct\xa01, 2017']
inc = inc20.copy()
inc['Oct 01, 2017'] = inc18['Oct\xa01, 2017'].values

In [ ]:
print(len(balsht18), len(balsht20))

31 36


In [ ]:
print(len(cf18), len(cf20))

38 45


In [ ]:
# Extracting information

#for profitability ratios 
revenue = earnings.loc['Total net revenues'][0]
cogs = earnings.loc['Product and distribution costs'][0]
operatingExpenses = earnings.loc['Total operating expenses'][0]
operatingIncome = earnings.loc['Operating income'][0] 
netIncome = earnings.loc['Net earnings attributable to Starbucks'][0]
eps = earnings.loc['Earnings per share — basic'][0]

# conver to millions * 1000000

# for liquidity ratios 
avg_assets = (balsht.loc['Total current assets'][0] + balsht.loc['Total current assets'][1])/2
avg_inv = (balsht.loc['Inventories'][0]+balsht.loc['Inventories'][0])/2
accountsPayable = balsht.loc['Accounts payable'][0]
currentAssets = balsht.loc['Total current assets'][0]
currentLiabilities = balsht.loc['Total current liabilities'][0]
prepaid_exp = balsht.loc['Prepaid expenses and other current assets'][0]

# for leverage ratios 
totalLiab = balsht.loc['Total liabilities'][0]
totalAssets = balsht.loc['TOTAL ASSETS'][0]
longTermDebt = balsht.loc['Long-term debt'][0]
totalStockholdersEquity = balsht.loc["Total shareholders’ deficit"][0]
interestExpenses = earnings.loc['Interest expense'][0]

#for activity ratios
inventory = balsht.loc['Inventories'][0]
accountsReceivable = balsht.loc['Accounts receivable, net'][0]
# averageDailySales = 


NameError: name 'balsht' is not defined

In [ ]:
#Profitability Ratios
gross_margin = (revenue - cogs)/revenue
operating_margin = operatingIncome/revenue
net_margin = netIncome/revenue
roe = netIncome/totalStockholdersEquity
roa = netIncome/totalAssets

#Efficiency Ratios
asset_turn = revenue/avg_assets
inv_turn = cogs/avg_inv
dso = (accountsReceivable/revenue)*365
dsi = (avg_inv/cogs)*365
dpo = (accountsPayable*365)/cogs
ccc = dso + dsi - dpo

#Liquidity ratios 
current_ratio = currentAssets/currentLiabilities
quick_ratio = (currentAssets - inventory - prepaid_exp)/currentLiabilities 
debt_equity = totalLiab/totalStockholdersEquity
# leverage = totalLiab/

#Growth ratio
prev_revenue = earnings.loc['Total net revenues'][1]
revenue_growth = (revenue - prev_revenue)/prev_revenue

In [ ]:
print('Profitability Ratios')
print(f'Revenue\t\t{revenue}')
print(f'Gross margin % \t{gross_margin*100:.2f} ')
print(f'Operating Income\t{operatingIncome}')
print(f'Operating Income Margin %\t{operating_margin*100:.2f}')
print(f'Net Income\t{netIncome}')
print(f'Net Margin %\t{net_margin:.2f}')
print(f'Return on Equity (ROE) %\t{roe*100:.2f}')
print(f'Return on Assets (ROA) %\t{roa*100:.2f}')
print(f'Earnings Per Share (EPS)\t{eps}')
print('\nEfficiency Ratios')
print(f'Asset Turnover\t{asset_turn:.2f}')
print(f'Inventory Turnover\t{inv_turn:.2f}')
print(f'Days Sales Outstanding\t{dso:.2f}')
print(f'Days Sales of Inventory\t{dsi:.2f}')
print(f'Payable Period\t{dpo:.2f}')
print(f'Cash Conversion Cycle\t{ccc:.2f}')
print('\nLiquidity & Financial Health Ratio\'s')
print(f'Current Ratio\t{current_ratio:.2f}')
print(f'Quick Ratio\t{quick_ratio:.2f}')
print(f'Debt/Equity\t{debt_equity:.2f}')
print(f'Financial Leverage')
print('\nYoY Growth %')
print(f'Revenue Growth %\t{revenue_growth:.2f}')


Profitability Ratios
Revenue		23518.0
Gross margin % 	67.28 
Operating Income	1561.7
Operating Income Margin %	6.64
Net Income	928.3
Net Margin %	0.04
Return on Equity (ROE) %	-11.89
Return on Assets (ROA) %	3.16
Earnings Per Share (EPS)	0.79

Efficiency Ratios
Asset Turnover	3.49
Inventory Turnover	4.96
Days Sales Outstanding	13.71
Days Sales of Inventory	73.59
Payable Period	47.33
Cash Conversion Cycle	39.97

Liquidity & Financial Health Ratio's
Current Ratio	1.06
Quick Ratio	0.75
Debt/Equity	-4.76
Financial Leverage

YoY Growth %
Revenue Growth %	-0.11


In [ ]:
# financials = stock.financials
# balanceSheet = stock.balance_sheet
# cashflow = stock.cashflow

In [ ]:
# # Extracting information

# #for profitability ratios 
# salesRevenue = financials.loc['Total Revenue'][0]
# cogs = financials.loc['Cost Of Revenue'][0]
# operatingExpenses = financials.loc['Total Operating Expenses'][0] 
# operatingIncome = financials.loc['Operating Income'][0]
# netIncome = cashflow.loc['Net Income'][0]

# # for liquidity ratios 
# currentAssets = balanceSheet.loc['Total Current Assets'][0]
# currentLiabilities = balanceSheet.loc['Total Current Liabilities'][0]

# # for leverage ratios 
# totalDebt = balanceSheet.loc['Total Liab'][0]
# totalAssets = balanceSheet.loc['Total Assets'][0]
# longTermDebt = balanceSheet.loc['Long Term Debt'][0]
# totalStockholdersEquity = balanceSheet.loc['Total Stockholder Equity'][0]
# operatingIncome = financials.loc['Operating Income'][0] 
# interestExpenses = financials.loc['Interest Expense'][0]

# #for activity ratios 
# inventory = balanceSheet.loc['Inventory'][0]
# # accountsReceivable = 
# # averageDailySales = 

In [ ]:
# # Profitability ratios 
# grossProfitMargin = (salesRevenue - cogs)/salesRevenue
# operatingProfitMargin = (salesRevenue - operatingExpenses)/salesRevenue
# netProfitMargin = netIncome/salesRevenue
# totalReturnOnAssets = netIncome/totalAssets
# returnOnStockHoldersEquity = netIncome/totalStockholdersEquity
# returnOnInvestedCapital = netIncome/(longTermDebt+totalStockholdersEquity)

In [ ]:
# # Liquidity ratios 
# currentRatio = currentAssets/currentLiabilities
# workingCapital = currentAssets - currentLiabilities

In [ ]:
# #Leverage ratios
# totalDebtToAssets = totalDebt/totalAssets
# longTermDebtToCapital = longTermDebt/(longTermDebt + totalStockholdersEquity)
# debtToEquity = totalDebt/totalStockholdersEquity
# longTermDebtToEquity = longTermDebt/totalStockholdersEquity
# coverage = operatingIncome/interestExpenses

In [ ]:
# #Activity Ratios 
# daysOfInventory = inventory / (cogs/365)
# inventoryTurnover = cogs/inventory
# # averageCollectionPeriod = accountsReceivable/averageDailySales

In [ ]:
# # Print ratios 
# print(f'{ticker} has the following ratios:')
# print('- Profitability ratios:')
# print(f'\tGross Profit Margin \t\t{grossProfitMargin:.2f}')
# print(f'\tOperating Profit Margin \t${operatingIncome:0,.2f}')
# print(f'\tNet Profit Margin \t\t{netProfitMargin:.2f}')
# print(f'\tReturn on Assets \t\t{totalReturnOnAssets:.2f}')
# print(f'\tReturn on Stockholders Equity \t{returnOnStockHoldersEquity:.2f}')
# print(f'\tReturn on Invested Capital \t{returnOnInvestedCapital:.2f}')

# print('- Liquidity ratios:')
# print(f'\tCurrent Ratio   \t\t{currentRatio:.2f}')
# print(f'\tWorking Capital  \t\t${workingCapital:0,.2f}')

# print('- Leverage ratios:')
# print(f'\tTotal debt-to-assets  \t\t{totalDebtToAssets:.2f}')
# print(f'\tLong-term debt-to-capital  \t{longTermDebtToCapital:.2f}')
# print(f'\tDebt-to-equity  \t\t{debtToEquity:.2f}')
# print(f'\tLong-term debt-to-equity  \t{longTermDebtToEquity:.2f}')
# print(f'\tTimes-interest-earned   \t{coverage:.2f}')

# print('- Activity ratios:')
# print(f'\tDays of Inventory  \t\t{daysOfInventory:.2f}')
# print(f'\tInventory Turnover  \t\t{inventoryTurnover:.2f}')


### Who holds the stock?

In [ ]:
# Who are the major holders? 
stock.major_holders

,0,1
0,0.17%,% of Shares Held by All Insider
1,71.11%,% of Shares Held by Institutions
2,71.24%,% of Float Held by Institutions
3,2801,Number of Institutions Holding Shares


In [ ]:
# Who are the insitutional holders?
insituational = stock.institutional_holders
insituational.sort_values(by='Value', ascending=False)

,Holder,Shares,Date Reported,% Out,Value
0,"Vanguard Group, Inc. (The)",96308860,2021-03-30,0.0818,10523669132
1,Blackrock Inc.,77169804,2021-03-30,0.0655,8432344483
2,State Street Corporation,46503246,2021-03-30,0.0395,5081409690
3,Magellan Asset Management Ltd,30087626,2021-03-30,0.0256,3287674893
4,"Geode Capital Management, LLC",19158097,2021-03-30,0.0163,2093405259
5,Northern Trust Corporation,18183318,2021-03-30,0.0154,1986891157
6,Bank of America Corporation,17417597,2021-03-30,0.0148,1903220824
7,"FMR, LLC",16139391,2021-03-30,0.0137,1763551254
8,"Loomis Sayles & Company, LP",14089525,2021-03-30,0.0120,1539562396
9,"Polen Capital Management, LLC",14048204,2021-03-30,0.0119,1535047251
